## Dependencies

In [ ]:
import requests
import pandas as pd
import json
import numpy as np
import csv
import time
import datetime
import sqlite3
import openpyxl

## Functions

In [ ]:


def axExtractDrugData(drug):
    """
    Given a drug, create a POST request to the web and capture the JSON. Then it transforms it into dataframe.

    Parameters:
    - drug (string): Substance that you want to look for.

    Return:
    dataframe: drug's dataframe

    Example:
    df=axExtractDrugData("urokinase")

    """

    # URL of the search page (pkdbai.com)
    url = 'https://pkpdai.com/_dash-update-component'

    # Build the payload with the drug search data
    payload = {
        "output": "datatable-interact.data",
        "outputs": {
            "id": "datatable-interact",
            "property": "data"
        },
        "inputs": [
            {
                "id": "button-db",
                "property": "n_clicks",
                "value": 5
            },
            {
                "id": "my-input-db",
                "property": "n_submit"
            }
        ],
        "changedPropIds": [
            "button-db.n_clicks"
        ],
        "state": [
            {
                "id": "my-input-db",
                "property": "value",
                "value": drug
            }
        ]
    }

    # Perform the POST request with the payload
    response = requests.post(url, json=payload)

    # Check if the response was successful
    if response.status_code == 200:
        # Filter the json
        json_data = response.json()
        dic_data = json_data['response']['datatable-interact']['data']
        dataframe_drug = pd.DataFrame(dic_data)

        # Check if df is empty (results as '')
        try:
            if dataframe_drug.iloc[0][1]=='':
                return None
        except:
            return None
        return dataframe_drug



    else:
        print("Failed to perform POST request")
        return -1




In [ ]:
def csv_to_comands_bbdd(file_csv, table_name):
    """
    Convert a CSV file into SQL commands for creating a table and inserting drug data.

    Parameters:
    - file_csv (str): The path to the CSV file.
    - table_name (str): The name of the table to be created.

    Returns:
    str: SQL commands for creating the table and inserting data.

    Example usage:
    >>> csv_to_comands_bbdd('data.csv', 'my_table')
    SQL commands string
    """
    with open(file_csv, 'r', encoding='UTF-8') as file:
        reader = csv.reader(file)
        header = next(reader)

        # Generate the CREATE TABLE command
        
        comand_create = f"""CREATE TABLE "{table_name}" (\n"""
        for column in header:
            comand_create += f"{column} VARCHAR(500),\n"
        comand_create = comand_create.rstrip(",\n")  # Remove the trailing comma and newline
        comand_create += "\n);\n"
        
        
        # Generate the INSERT INTO commands
        comands_insert = []
        for row in reader:
            values = []
            for value in row:
                if isinstance(value, str):
                    value = value.replace("'", "''")  # Duplicate single quotes
                    value = f"'{value}'"  # Enclose in single quotes
                values.append(str(value))
            values_str = ", ".join(values)
            comand_insert = f"""INSERT INTO "{table_name}" VALUES ({values_str});\n"""
           
            comands_insert.append(comand_insert)
            

    # Combine the CREATE TABLE and INSERT INTO commands into a single string
    comands_bbdd = comand_create + "\n".join(comands_insert)
    return comands_bbdd

In [ ]:
def create_files_for_drug_list(drug_list):
  """
    Create files and store data in a database for a list of drugs.

    Parameters:
    - drug_list (list): A list of dictionaries containing drug information with 'name' and 'drugbank_id' keys.

    Returns:
    None

    Example usage:
    >>> create_files_for_drug_list(drugs)
  """
  conn=sqlite3.connect("DRUG_DB.db")
  
  cursor=conn.cursor()
  try:
    

    # Set up the error log file
    with open(f'errors{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.log', 'w') as file:
      cont=0
      total=len(drug_list)
      for drug in drug_list:
        df=axExtractDrugData(drug['name'])
        if isinstance(df, pd.DataFrame):
          # IMPORTANT -> For futures searchs, escape these characters
          # name = drug['name']
          # name = name.replace(' ', '_')
          # name = name.replace('-', '_')
          # name = name.replace('(', '')
          # name = name.replace(')', '')
          # name = name.replace(',', '_')
          df['Drugbank_id']=drug['drugbank_id']
          df['Name']=drug['name']
          df.to_csv(drug['name']+".csv", index=False)
          comands=csv_to_comands_bbdd(drug['name']+".csv",drug['name'])
          # Execute the SQL commands
          try: 
      
            cursor.executescript(comands)
            
          except sqlite3.Error as error:
            
            print(f"NOT AN ERROR: {str(error)}.In Drug NAME {drug['name']}")
            file.write(f"\nNOT AN ERROR: {str(error)}")
            # Rollback changes in case of error
            conn.rollback()
            cont=cont+1
          else:
            conn.commit()
            print(f"{drug['name']} saved successfully!")
            cont=cont+1
            time.sleep(17)
        elif df == -1:
          file.write(f"\nFailed to perform POST request in Drug NAME {drug['name']} with ID {drug['drugbank_id']}")
          print(f"Failed to perform POST request in Drug NAME {drug['name']} with ID {drug['drugbank_id']}")
        elif df == None:
          file.write(f"\nDrug NAME {drug['name']} with ID {drug['drugbank_id']}  was not found")
          print(f"Drug NAME {drug['name']} with ID {drug['drugbank_id']}  was not found")
 
  finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()
    total=(cont/total)*100
    print("----------------------DONE-----------------------")
    print(f"•	Saved: {total} %")




## Main

In [ ]:

df = pd.read_excel('20230629_drugbank_drugs-NAME_ID.xlsx', engine='openpyxl')
drug_list = []
for index, row in df.iterrows():
    name = row['GENERIC_NAME']
    drugbank_id = row['DRUGBANK_ID']
    
    drug_list.append({'name':name, 'drugbank_id': drugbank_id})
create_files_for_drug_list(drug_list)

